In [1]:
from __future__ import print_function, division
from gurobipy import Model, GRB, quicksum, LinExpr
import networkx as nx
import pickle
import numpy, random

In [2]:
def MIP_IM():
    budget = 25
    m = 100
    for index in range(20):
        with open('../MIP/data/networks_prob/graph_spa_500_'+str(index)+'.pickle', "rb") as f:
            main_graph = pickle.load(f)
            
            samples = []
            for j in range(m):
                G = nx.DiGraph()
                for u in main_graph.nodes():
                    G.add_node(u)
                for u,v in main_graph.edges():
                    if main_graph[u][v]['p']> random.random():
                        G.add_edge(u, v)
                samples.append(G)
                
                
            model = Model('maximin_'+str(index))
            min_value = model.addVar(lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS)

            mvars = []
            #active nodes
            avars = []
            #seed nodes
            svars = []
            var_seed_dict = {}
            var_active_dict = {}
            var_mean_dict = {}

            for j in range(len(main_graph.nodes())):
                s = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                svars.append(s)
                var_seed_dict[j] = s

            for sample_index, sample in enumerate(samples):
                for j in range(len(main_graph.nodes())):
                    a = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                    avars.append(a)
                    var_active_dict[(sample_index,j)] = a    

            mvars.append(avars)
            mvars.append(svars)
            
            budget = 25
            model.addConstr(quicksum(svars), GRB.LESS_EQUAL, budget)
            
            obj_expr = quicksum(avars)
            model.setObjective(min_value, GRB.MAXIMIZE)
            
            for sample_index, sample in enumerate(samples):
                for i in range(len(main_graph.nodes())):
                    neighbors = sample.in_edges(i) 
                    e = len(neighbors)
                    ai = var_active_dict[(sample_index,i)]
                    si = var_seed_dict[i]
                    if i in  var_mean_dict.keys():
                        var_mean_dict[i]+= ai
                    else:
                        var_mean_dict[i]= ai
                    neighbors_vars = []
                    for neighbor in neighbors:
                        neighbors_vars.append(var_active_dict[(sample_index,neighbor[0])])
                    expr = (e+1)*ai-si-quicksum(neighbors_vars)
                    model.addConstr(expr, GRB.GREATER_EQUAL, 0)
                    model.addConstr(expr, GRB.LESS_EQUAL, e)
            for i in range(len(main_graph.nodes())):
                model.addConstr(var_mean_dict[i], GRB.GREATER_EQUAL, m* min_value)
                
            try:
                model.optimize()
            except e:
                print(e)
                
            with open('../Git/influence_maximization/experiments/im500/results/fairmip/maxmin/base100/output_'+str(index)+'.txt', "w") as of:    
                for key,value in var_seed_dict.items():
                    if(value.x > 0):
                        print(key, file=of)

In [3]:
MIP_IM()

Optimize a model with 100501 rows, 50501 columns and 299346 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 64116 rows and 32126 columns
Presolve time: 1.58s
Presolved: 36385 rows, 18375 columns, 131348 nonzeros
Variable types: 0 continuous, 18375 integer (18374 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 600 rows and 302 columns
Presolved: 35785 rows, 18073 columns, 129082 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   13779    6.1169797e-02   0.000000e+00   2.073984e+00      5s
   18035    8.9937172e-02   0.000000e+00   4.160269e+00     10s
   20723    1.0859397e-01   0.000000e+00   3.427405e+00     15s
   22739    1.18232

H    0     0                       0.0100000    0.12176  1118%     -   20s
     0     0    0.10669    0 12237    0.01000    0.10669   967%     -   32s
H    0     0                       0.0200000    0.10669   433%     -   33s
     0     0    0.10455    0 11992    0.02000    0.10455   423%     -   34s
     0     0    0.09078    0 9853    0.02000    0.09078   354%     -   42s
H    0     0                       0.0300000    0.09078   203%     -   42s
     0     0    0.08810    0 9526    0.03000    0.08810   194%     -   43s
     0     0    0.08465    0 9692    0.03000    0.08465   182%     -   44s
     0     0    0.08443    0 9596    0.03000    0.08443   181%     -   44s
     0     0    0.08096    0 9436    0.03000    0.08096   170%     -   45s
     0     0    0.08087    0 9504    0.03000    0.08087   170%     -   45s
     0     0    0.07737    0 9529    0.03000    0.07737   158%     -   46s
     0     0    0.07716    0 9482    0.03000    0.07716   157%     -   47s
     0     0    0.07487

     0     0    0.10947    0 13996    0.05000    0.10947   119%     -   80s
     0     0    0.10765    0 13858    0.05000    0.10765   115%     -   81s
     0     0    0.10733    0 14007    0.05000    0.10733   115%     -   82s
     0     0    0.10548    0 14258    0.05000    0.10548   111%     -   83s
     0     0    0.10523    0 14091    0.05000    0.10523   110%     -   84s
     0     0    0.10451    0 14359    0.05000    0.10451   109%     -   85s
     0     0    0.10441    0 14427    0.05000    0.10441   109%     -   86s
     0     0    0.10319    0 14255    0.05000    0.10319   106%     -   87s
     0     0    0.10308    0 14313    0.05000    0.10308   106%     -   88s
     0     0    0.10164    0 14574    0.05000    0.10164   103%     -   89s
     0     0    0.10153    0 14422    0.05000    0.10153   103%     -   90s
     0     0    0.10026    0 14265    0.05000    0.10026   101%     -   91s
     0     0    0.10017    0 14307    0.05000    0.10017   100%     -   92s
     0     0

     0     0    0.07565    0 11586    0.04000    0.07565  89.1%     -   73s
     0     0    0.07553    0 12070    0.04000    0.07553  88.8%     -   74s
     0     0    0.07539    0 12019    0.04000    0.07539  88.5%     -   76s
     0     0    0.07538    0 11988    0.04000    0.07538  88.4%     -   76s
     0     0    0.07525    0 11844    0.04000    0.07525  88.1%     -   77s
     0     0    0.07525    0 11741    0.04000    0.07525  88.1%     -   79s
     0     2    0.07525    0 11741    0.04000    0.07525  88.1%     -   83s
     3     4    0.07000    2 8754    0.04000    0.07000  75.0%  2383   85s
     9    12    0.07000    4 9504    0.04000    0.07000  75.0%  4005   93s
    13    15    0.07000    5 9049    0.04000    0.07000  75.0%  4816   97s
    21    19    0.07000    6 9255    0.04000    0.07000  75.0%  4726  102s
    29    26    0.07000    7 10238    0.04000    0.07000  75.0%  4096  105s
    37    36    0.06845    8 10205    0.04000    0.07000  75.0%  3744  110s
    44    40    

     0     0    0.05881    0 9036    0.04000    0.05881  47.0%     -   68s
     0     0    0.05775    0 8546    0.04000    0.05775  44.4%     -   69s
     0     0    0.05762    0 8905    0.04000    0.05762  44.0%     -   69s
     0     0    0.05760    0 8813    0.04000    0.05760  44.0%     -   70s
     0     0    0.05760    0 8343    0.04000    0.05760  44.0%     -   72s
     0     1    0.05760    0 8341    0.04000    0.05760  44.0%     -   74s
     8     8    0.05000    4 1362    0.04000    0.05000  25.0%   756   75s

Cutting planes:
  Cover: 3175
  Clique: 3651
  MIR: 263
  StrongCG: 3
  Zero half: 151

Explored 50 nodes (82930 simplex iterations) in 76.56 seconds
Thread count was 8 (of 8 available processors)

Solution count 5: 0.04 0.03 0.02 ... -0

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e-02, best bound 4.000000000000e-02, gap 0.0000%
Optimize a model with 100501 rows, 50501 columns and 299918 nonzeros
Variable types: 1 continuous, 50500 integer 

...

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 64046 rows and 32100 columns
Presolve time: 1.86s
Presolved: 36455 rows, 18401 columns, 131330 nonzeros
Variable types: 0 continuous, 18401 integer (18400 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 865 rows and 438 columns
Presolved: 35590 rows, 17963 columns, 128063 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   13293    6.0499074e-02   0.000000e+00   1.777564e+00      5s
   17885    8.2316927e-02   0.000000e+00   5.700057e-01     10s
   21245    1.0330357e-01   0.000000e+00   2.033303e+00     15s
   23485    1.1337823e-01   0.000000e+00   8.531503e+01     20s
Concurrent spin time: 1.54s

Solved with dual simplex

Root relaxation: objective 1.310335e-01, 27812 iterations, 19.97 seconds

    Nodes    |    Current Node    |     Object

     0     0    0.09282    0 10641    0.04000    0.09282   132%     -   64s
     0     0    0.09245    0 10786    0.04000    0.09245   131%     -   65s
     0     0    0.09078    0 10806    0.04000    0.09078   127%     -   66s
     0     0    0.09071    0 10856    0.04000    0.09071   127%     -   66s
     0     0    0.09016    0 11092    0.04000    0.09016   125%     -   67s
H    0     0                       0.0500000    0.09016  80.3%     -   67s
     0     0    0.08960    0 10735    0.05000    0.08960  79.2%     -   68s
     0     0    0.08881    0 10870    0.05000    0.08881  77.6%     -   69s
     0     0    0.08871    0 10936    0.05000    0.08871  77.4%     -   69s
     0     0    0.08778    0 10890    0.05000    0.08778  75.6%     -   70s
     0     0    0.08755    0 10852    0.05000    0.08755  75.1%     -   70s
     0     0    0.08667    0 11409    0.05000    0.08667  73.3%     -   72s
     0     0    0.08662    0 11238    0.05000    0.08662  73.2%     -   72s
     0     0 

     0     0    0.10324    0 11978    0.04000    0.10324   158%     -   78s
H    0     0                       0.0500000    0.10324   106%     -   78s
     0     0    0.10134    0 11657    0.05000    0.10134   103%     -   78s
     0     0    0.09864    0 11322    0.05000    0.09864  97.3%     -   80s
     0     0    0.09840    0 11517    0.05000    0.09840  96.8%     -   81s
     0     0    0.09682    0 11379    0.05000    0.09682  93.6%     -   82s
     0     0    0.09670    0 11442    0.05000    0.09670  93.4%     -   82s
     0     0    0.09623    0 11698    0.05000    0.09623  92.5%     -   84s
     0     0    0.09612    0 11637    0.05000    0.09612  92.2%     -   84s
     0     0    0.09534    0 11623    0.05000    0.09534  90.7%     -   85s
     0     0    0.09528    0 11779    0.05000    0.09528  90.6%     -   86s
     0     0    0.09315    0 11661    0.05000    0.09315  86.3%     -   87s
     0     0    0.09277    0 11635    0.05000    0.09277  85.5%     -   88s
     0     0 

     0     0    0.08484    0 12672    0.05000    0.08484  69.7%     -   83s
     0     0    0.08467    0 12574    0.05000    0.08467  69.3%     -   87s
     0     0    0.08467    0 12574    0.05000    0.08467  69.3%     -   88s
     0     2    0.08467    0 12574    0.05000    0.08467  69.3%     -   93s
     3     4    0.08000    2 9817    0.05000    0.08000  60.0%  6908  101s
     5     8    0.08000    3 8849    0.05000    0.08000  60.0%  5150  106s
     9    12    0.08000    4 9976    0.05000    0.08000  60.0%  5703  112s
    13    16    0.08000    4 10397    0.05000    0.08000  60.0%  6409  118s
    17    20    0.08000    5 10210    0.05000    0.08000  60.0%  6828  122s
    21    22    0.08000    5 9560    0.05000    0.08000  60.0%  6335  125s
    30    31    0.07882    7 10814    0.05000    0.08000  60.0%  5440  133s
    35    33    0.08000    8 10050    0.05000    0.08000  60.0%  5070  136s
    43    40    0.08000    9 10242    0.05000    0.08000  60.0%  4539  143s
    59    58    

Thread count was 8 (of 8 available processors)

Solution count 6: 0.05 0.04 0.03 ... -0

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e-02, best bound 5.000000000000e-02, gap 0.0000%
Optimize a model with 100501 rows, 50501 columns and 299642 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 64205 rows and 32179 columns
Presolve time: 1.90s
Presolved: 36296 rows, 18322 columns, 130785 nonzeros
Variable types: 0 continuous, 18322 integer (18321 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 392 rows and 198 columns
Presolved: 35904 rows, 18124 columns, 129322 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
